In [ ]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from translate import Translator

: 

In [ ]:
coursera=  pd.read_csv("data/Coursera.csv")
edx= pd.read_csv("data/EdX.csv")
udacity= pd.read_csv("data/Udacity.csv")

coursera['University'] = coursera['University'].apply(lambda x: 'É' + x[1:] if x.startswith('�') else 'é' + x[1:] if '�' in x else x)
coursera['University'] = coursera['University'].replace('École Polytechnique F�d�rale de Lausanne', 'École Polytechnique Fédérale de Lausanne')
# Add a new column named "Website" with the value "Coursera" in each row
coursera.insert(0, "Website", "Coursera")
edx.insert(0, "Website", "EdX")
udacity.insert(0, "Website", "Udacity")

coursera.head(5)

: 

In [ ]:
# Rename the "Course Type" column to "Type of Course"
coursera = coursera.rename(columns={"Course URL": "Link"})
coursera = coursera.rename(columns={"University": "Institution"})
coursera = coursera.rename(columns={"Course Name": "Name"})
coursera['Course Description'] = coursera['Course Description'] + coursera['Skills']
# Keeping important columns for recommendation
coursera = coursera[['Website','Name','Institution', 'Difficulty Level', 'Course Rating','Link','Course Description']]
coursera.head(5)

: 

In [ ]:
coursera.shape

: 

In [ ]:
edx = edx.rename(columns={"University": "Institution"})
edx['Course Description'] = edx['Course Description'] + edx['About']
# Keeping important columns for recommendation
edx = edx[['Website','Name','Institution','Difficulty Level','Link','Course Description']]
edx.head(5)

: 

In [ ]:
edx.shape

: 

In [ ]:
udacity = udacity.rename(columns={"School": "Institution"})
udacity = udacity.rename(columns={"Rating": "Course Rating"})
udacity = udacity.rename(columns={"About": "Course Description"})
udacity.head(5)

: 

In [ ]:
edx.shape

: 

In [ ]:
coursera.duplicated().sum()

: 

In [ ]:
coursera.drop_duplicates(inplace=True)

: 

In [ ]:
coursera.duplicated().sum()

: 

In [ ]:
udacity.duplicated().sum()

: 

In [ ]:
edx.duplicated().sum()

: 

In [ ]:
edx.drop_duplicates(inplace=True)

: 

In [ ]:
edx.duplicated().sum()

: 

: 

In [ ]:
# Concatenate the datasets vertically
merged_data = pd.concat([coursera, edx, udacity], ignore_index=True)
merged_data.head(5000)


: 

In [ ]:
merged_data.duplicated().sum()

: 

In [ ]:
import nltk
from nltk.tokenize import word_tokenize


# Tokenize function
def tokenize_description(description):
    if isinstance(description, str):
        return word_tokenize(description)
    else:
        return []

# Apply tokenization and create a new column
merged_data['Description'] = merged_data['Course Description']    
# .apply(tokenize_description)

# Reorder the columns (optional)
merged_data = merged_data[['Website', 'Name', 'Institution', 'Difficulty Level', 'Link', 'Description']]

: 

In [ ]:
merged_data.head(5)

: 

In [ ]:
# merged_data['Name'] = merged_data['Name'].apply(lambda x:x.split())
merged_data['Institution'] = merged_data['Institution'].apply(lambda x:x.split())
merged_data['Difficulty Level'] = merged_data['Difficulty Level']
# merged_data.sample(4)

: 

In [ ]:
# merged_data.iloc[0]['Name']

: 

In [ ]:
merged_data["combined"]= merged_data["Institution"]+merged_data["Description"].apply(tokenize_description)
merged_data.head()

: 

In [ ]:
# Converting list to str
merged_data['combined'] = merged_data['combined'].apply(lambda x: " ".join(x))

merged_data.head()

: 

In [ ]:
# Converting to lower case
merged_data['combined'] = merged_data['combined'].apply(lambda x:x.lower())


: 

In [ ]:
merged_data.iloc[0]['combined']

: 

In [ ]:
import nltk
from nltk.stem import PorterStemmer

: 

In [ ]:
ps = PorterStemmer()

: 

In [ ]:
def stems(text):
    T = []
    
    for i in text.split():
        T.append(ps.stem(i))
    
    return " ".join(T)

: 

In [ ]:
merged_data['combined'] = merged_data['combined'].apply(stems)
merged_data.iloc[0]['combined']

: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

: 

In [ ]:
vector = cv.fit_transform(merged_data['combined']).toarray()

: 

In [ ]:
vector[0]

: 

In [ ]:
vector.shape

: 

In [ ]:
len(cv.get_feature_names())

: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

: 

In [ ]:
similarity = cosine_similarity(vector)

: 

In [ ]:
similarity.shape

: 

In [ ]:
# similarity
merged_data[merged_data['Name'] == 'Finance for Managers'].index[0]

: 

In [ ]:
def recommend(course):
    # Convert the input course to lowercase
    course = course.lower()
    # Get the index of the course
    index = merged_data[merged_data['Name'].str.lower().str.contains(course)].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(merged_data.iloc[i[0]].Name)

: 

In [ ]:
recommend('Finance')

: 

In [ ]:
import pickle as pickle


: 

In [ ]:
# pickle.dump(merged_data,open('artifacts/final_data.pkl','wb'), protocol=4)
# pickle.dump(similarity,open('artifacts/similarity.pkl','wb'), protocol=4)
# with open('artifacts/final_data.pkl', 'wb') as f:
#     pickle.dump(merged_data, f, protocol=5)

# with open('artifacts/similarity.pkl', 'wb') as f:
#     pickle.dump(similarity, f, protocol=5)

: 

In [ ]:
merged_data.to_csv('artifacts/final_data.csv', index=False)
pd.DataFrame(similarity).to_csv('artifacts/similarity.csv', index=False)

: 